In [ ]:
###INDIVIDUAL SENSOR DURATION###

#dependencies
import numpy as np
import matplotlib.pyplot as plt 
import os,sys
import pylab
import h5py
import matplotlib as mpl
import pandas as pd
import hvplot.pandas
import hvplot.dask

#variables
directory = '/directory/file'
fps = '.'
allSensor = []

#create dataframe shell
#list sensor numbers
df = pd.DataFrame({'sensor number' : allSensor})
df.set_index('sensor number')

#function to identify 1 location in array 
def one_runs(a):
    # Create an array that is 1 where a is 1, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 1).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

#read files from directory 
ColumnNames = ['sensor number']
i_file = 0;
for filename in sorted(os.listdir(directory)):
    if filename.endswith(".mat"): 
        i_file = i_file + 1;
        name = os.path.join(directory, filename)  
        
        #read mat7.3 file and clean up 
        f = h5py.File(name, 'r')

        trialName = name[-5]
        ColumnNames.append(trialName)

        Sensor = []

        #shape is approximating number frames 
        struArray = f['data/frame']
        shape = struArray['sns'].shape
        shape = shape[0] - 1 

        while shape > 0: 
            value = f[struArray['sns'][shape,0]].value 
            Sensor.append(value)
            shape = shape - 1 
            
        #remove extraneous array bracket 
        frames = np.array([e[0] for e in Sensor])

        #find location of 1 runs
        OnesLoc = []
        for array in frames.T: 
            OnesLoc.append(one_runs(array))

        #remove never active sensors (empty arrays) 
        #parse out reward port touch into separate array 
        #reward port touch not in use in this code
        touches = []
        sensorID = []
        lick1 = []
        lick2 = []
        for i,v in enumerate(OnesLoc):
            if i == 0: #reward port 1 hooked up to 0 
                lick1.append(v)
            elif i == 94: #reward port 2 hooked up 94
                lick2.append(v)
            elif len(v) != 0:  
                touches.append(v/fps)
                sensorID.append(i)
            else: 
                pass

        #save max length of sensor touch for each sensor   
        maxDuration = []      
        for array in touches: 
            diffs = []
            for array in array: 
                diffs.append(np.diff(array))
            maxDuration.append(float(max(diffs)))
                    
        #add new column of max sensor duration for each sensor to df 
        df2 = pd.DataFrame({'sensor number' : transform, i_file : maxDuration})
        df = df.merge(df2, on='sensor number', how = 'outer')
        df = df.sort_values(by=['sensor number'])
        print("...completed " + str(i_file))

        

#transpose df for graphing purposes 
df2 = df1.T 
#remove extraneous index on row from transposing and make sensor ID the index 
df2.columns = df2.iloc[0]
df2.drop(df2.index[:1], inplace=True)
#get mean of max sensor duration 
Mean = df2.mean(axis=1)


#save table
#df2.to_hdf("name.h5", 'df', format = 'table')